In [ ]:
%pylab inline
plt.style.use("default")
import os, sys
import scipy
import random
import pandas as pd
import helpers as h
import spontHelpers as sh
import matplotlib.pyplot as plt
import utils as u
import ctils as cx
import matplotlib as mpl
from glob import glob
from scipy.io import loadmat
from scipy.stats import skew, kurtosis
from sklearn.cluster import KMeans
from mpl_toolkits import mplot3d
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

plt.rcParams['figure.figsize'] = (15, 10) 

In [ ]:
def clean_unpack(data0):
    """returns clean and zscored signal"""
    resp0, spon0, istim0 = h.unbox(data0)

    resp0 = h.denoise_resp(resp0, spon0)

    resp0 = h.dupSignal(resp0, istim0)

    return resp0, spon0


In [ ]:
def get_subpops(x, y, dx, dy, resp):

    inxs = []
    subpops = []
    xmax = x.max()
    ymax = y.max()
    Xrange = np.arange(xmax, step=dx)
    Yrange = np.arange(ymax, step=dy)
    for x0 in Xrange:
        for y0 in Yrange:
            x1 = x0+dx
            y1 = y0+dx
            inx = np.where(((x>x0) & (x<x1)) & ((y > y0) & (y < y1)))
            inxs.append(inx)
            subpop = resp[:, :, inx].squeeze(axis=2)
            subpops.append((((x0, x1), (y0, y1)), subpop))
    return subpops, inxs 


In [ ]:
def compute_cvPCA(resp0, nshuff, maxcols=np.inf):
    ss0 = u.shuff_cvPCA(resp0, nshuff=nshuff, maxcols=maxcols)

    ss0 = ss0.mean(axis=0)
    ss0 = ss0 / ss0.sum()

    return ss0

def compute0_cvPCA(resp0, nshuff, maxcols=100):
    ss0 = cx.shuff_cvPCA(resp0, nshuff=nshuff, maxcols=maxcols)

    ss0 = ss0.mean(axis=0)
    ss0 = ss0 / ss0.sum()

    return ss0

def vanilaPCA(resp0):
    ss0, sv = u.vanila_PCA(resp0)
    ss0 = ss0/ss0.sum()
    sv = sv/sv.sum()

    return ss0, sv


In [ ]:
data_root = "/camp/home/duuta/working/duuta/ppp0/data/stringer/live_data/"
data_files = [fname.split("/")[-1] for fname in glob(f"{data_root}natimg2800_M*.mat")]



In [ ]:
maxstim0 = spontList[1]
maxstim1 = spontList[3]


In [ ]:
maxstim_res0 = respList[1]
maxstim_res1 = respList[3]


In [ ]:
# this computation for alpha values for response
ra0 = []
ra1 = []
nstims = np.linspace(120, 2300, 100).astype('int')
for ns in nstims:
    r0 = maxstim_res0[:ns, :]
    r1 = maxstim_res1[:ns, :]
    sr0 = compute0_cvPCA(r0, 10, maxcols=100)
    sr1 = compute0_cvPCA(r1, 10, maxcols=100)
    ras0, _, _ = cx.get_powerlaw(sr0, np.arange(5, 90).astype('int'))
    ras1, _, _ = cx.get_powerlaw(sr1, np.arange(5, 90).astype('int'))
    ra0.append(ras0)
    ra1.append(ras1)


In [ ]:
plt.plot(nstims, ra0, label='ani0')
plt.plot(nstims, ra1, label='ani1')
plt.xlabel("Stimulus size", fontsize=20)
plt.ylabel("Alpha Value", fontsize=20)
plt.title("resp:Value of alpha for differing size of stimulus", fontsize=20)
plt.legend(fontsize=20)


In [ ]:
# change the end points of stimulus 
sppa0 = []
sppa1 = []
nstims = np.linspace(120, 2500, 100).astype('int')
for ns in nstims:
    ps0 = sh.ssplit(maxstim0[:ns, :])
    ps1 = sh.ssplit(maxstim1[:ns, :])
    ts0 = compute_cvPCA(ps0, 10, maxcols=100)
    ts1 = compute_cvPCA(ps1, 10, maxcols=100)
    aps0, _, _ = u.get_powerlaw(ts0, np.arange(5, 90).astype('int'))
    aps1, _, _ = u.get_powerlaw(ts1, np.arange(5, 90).astype('int'))
    sppa0.append(aps0)
    sppa1.append(aps1)


In [ ]:
plt.plot(nstims, sppa0, label='ani0')
plt.plot(nstims, sppa1, label='ani1')
plt.xlabel("Stimulus size", fontsize=20)
plt.ylabel("Alpha Value", fontsize=20)
plt.title("Value of alpha for differing size of stimulus", fontsize=20)
plt.legend(fontsize=20, loc='upper left')


In [ ]:
# use all the components to make clear the changes.
# add arrs for alpha values.
plt.figure(figsize=(20, 20))
fig0, axs = plt.subplots(nrows=6, ncols=2, layout='constrained', figsize=(10, 10))
for i, arr in enumerate(respList):
    rr0_sn = compute_cvPCA(arr, 10, maxcols=np.inf)
    _, rr0_so = vanilaPCA(arr)
    a0_sn, yp0, _ = u.get_powerlaw(rr0_sn, np.arange(11, 500).astype('int'))
    a0_so, yp1, _ = u.get_powerlaw(rr0_so, np.arange(11, 90).astype('int'))
    axs[i, 0].loglog(np.arange(0, rr0_sn.size), rr0_sn/rr0_sn.sum())
    axs[i, 0].loglog(np.arange(0, rr0_sn.size), yp0)
    axs[i, 0].text(x=10**2.5, y=10**-2, s=f"alpha={round(a0_sn, 2)}")
    axs[i, 0].set_title("nshuff on")
    axs[i, 0].set_ylabel("Variance")
    
    axs[i, 1].loglog(np.arange(0, rr0_so.size), rr0_so/rr0_so.sum(), label='Shuff off')
    axs[i, 1].loglog(np.arange(0, rr0_so.size), yp1)
    axs[i, 1].text(x=10**2.5, y=10**-2.5, s=f"alpha={round(a0_so, 2)}")
    axs[i, 1].set_title("nshuff off")
    axs[i, 1].set_xlabel("Dimensions")
        
    fig0.suptitle('resp')


In [ ]:
nwsList = []
nstim = 280
for arr in spontList:
    sp0 = arr[:nstim, :]
    nwsList.append(sp0)


In [ ]:
nrois = 9000
nrois_stims = []
for arr in spontList:
    sp00 = arr[:nstim, :nrois]
    nrois_stims.append(sp00)


In [ ]:
aa0= []
for arr in nrois_stims:
    sa0 = sh.ssplit(arr)
    sa1 = compute_cvPCA(sa0, 100, maxcols=100)
    av, _, _ = u.get_powerlaw(sa1, np.arange(11, 90).astype('int'))
    aa0.append(av)


In [ ]:

plt.bar(list(np.arange(6)), aa0)
plt.title('alpha (nshuff=100) for fixed shape arr (280, 9000) for six different mice', fontsize=20)
plt.xlabel("different mice", fontsize=20)
plt.ylabel("Val of alpha", fontsize=20)


In [ ]:
ass = []
kiks = []
for arr in nwsList:
    ss = sh.ssplit(arr)
    kiks.append(arr.shape[1])

    ss0 = compute_cvPCA(ss, 10,  maxcols=100)
    a, _, _ = u.get_powerlaw(ss0, np.arange(11, 90).astype('int'))
    ass.append(a)


In [ ]:
plt.plot(kiks, ass, '.')
plt.title("for fixed stimulus size, the difference in alpha for diferring sizes of neurons, but fixed components explained.", fontsize=20)
plt.xlabel("# of neurons", fontsize=20)
plt.ylabel("Alpha Value", fontsize=20)
plt.grid(True)

# kept stimulus size fixed but #neruons & ncomponents fixed.


In [ ]:
tas = []
pps = []
for arr in nrois_stims:
    t0 = sh.ssplit(arr)
    t1, tv = vanilaPCA(s0)
    tt, _, _ = u.get_powerlaw(t1, np.arange(11, 90).astype('int'))
    tp, _, _ = u.get_powerlaw(tv, np.arange(11, 90).astype('int'))
    tas.append(tt)
    pps.append(tp)



In [ ]:
# difference in alpha values across different sizes of neurons.
plt.scatter(list(np.arange(6)), tas)
plt.title('Alpha values computed from explained variance for vanila PCA')
plt.xlabel('different mice')
plt.ylabel('Value of alpha')


In [ ]:
plt.scatter(list(np.arange(6)), pps)
plt.title('Alpha values for singular values vanila PCA')
plt.xlabel('different mice')
plt.ylabel('Value of alpha')
